In [1]:
import pandas as pd
import numpy as np

In [3]:
markup = pd.read_csv('markup.csv', sep = ';')
valid = pd.read_csv('valid.csv', sep = ';')

In [4]:
markup = pd.read_csv('markup.csv', sep = ';')

In [5]:
valid

,text_id,subject1,subject2,subject3,subject4,ans_set
0,text_id1,mq3,mq2,mq2,mq3,NaN
1,text_id2,NaN,NaN,NaN,NaN,2
2,text_id4,NaN,NaN,NaN,NaN,1;3


# Делаем обязательную часть из условия задачи:
1. переводим markup_dt в datetime
2. проставляем порядковый номер ans_set для ответа размечтика

In [6]:
# присваиваем ans_set

markup['markup_dt'] = pd.to_datetime(markup['markup_dt'])  # трансформируем в нужный тип данных
markup.sort_values(by=['text_id', 'markup_dt'], ascending=[True, True], 
                   inplace=True)  # сортируем по каждому тексту ответы разметчиков по времени
markup['ans_set'] = markup.groupby(['text_id'])['markup_dt'].rank()  # присваиваем ans_set от 1 до 3 для каждого текста
markup['ans_set'] = markup['ans_set'].astype(int)

# Решаем задачу №1 - создать витрину правильных ответов по каждому тексту, который пойдет на разметку

In [7]:
# создаем датасет, куда будем складывать правильные ответы

markup_answers = pd.DataFrame()
texts = list(set(markup['text_id'].values))
markup_answers['text_id'] = texts

In [8]:
markup_answers

,text_id
0,text_id2
1,text_id1
2,text_id4
3,text_id3


In [9]:
markup_answers.sort_values(by='text_id', inplace=True)  # сортируем в нужном порядке текста

In [10]:
markup_answers['subject1'] = markup.groupby('text_id')['subject1'].agg(
    lambda x: x.value_counts().index[0] 
    if x.value_counts().values.max() >= 2 else np.nan
).values  # порядок соответсвует ответам, потому что изначальный датафрейм markup мы отсортировали по текстам

In [11]:
markup_answers['subject2'] = markup.groupby('text_id')['subject2'].agg(
    lambda x: x.value_counts().index[0] 
    if x.value_counts().values.max() >= 2 else np.nan
).values

In [12]:
markup_answers['subject3'] = markup.groupby('text_id')['subject3'].agg(
    lambda x: x.value_counts().index[0] 
    if x.value_counts().values.max() >= 2 else np.nan
).values

In [13]:
markup_answers['subject4'] = markup.groupby('text_id')['subject4'].agg(
    lambda x: x.value_counts().index[0] 
    if x.value_counts().values.max() >= 2 else np.nan
).values

In [14]:
markup_answers  # получен ли консенсус по текстам и subject, если есть консенсус по всем subject, то не будет nan в строке по тексту

,text_id,subject1,subject2,subject3,subject4
1,text_id1,mq2,mq2,NaN,mq3
0,text_id2,NaN,mq3,mq1,mq1
3,text_id3,mq2,mq3,mq1,mq1
2,text_id4,NaN,mq3,mq1,mq1


In [15]:
consensus = markup_answers.dropna()

In [16]:
consensus # те тексты по которым был консенсус по всем subject

,text_id,subject1,subject2,subject3,subject4
3,text_id3,mq2,mq3,mq1,mq1


In [17]:
consensus_valid = pd.concat([valid, consensus])

In [18]:
consensus_valid.reset_index(drop=True, inplace=True)

In [19]:
consensus_valid # датафрейм который состоит из valid и тех строк где получен консенсус

,text_id,subject1,subject2,subject3,subject4,ans_set
0,text_id1,mq3,mq2,mq2,mq3,NaN
1,text_id2,NaN,NaN,NaN,NaN,2
2,text_id4,NaN,NaN,NaN,NaN,1;3
3,text_id3,mq2,mq3,mq1,mq1,NaN


In [20]:
final = markup.merge(consensus_valid, how='left', on='text_id', suffixes=['_mark', '_valid']) 

In [21]:
# проверка на то указал ли валидатор несколько значений или нет: если несколько взять меньший
final['ans_set_valid'] = np.where(
    len(final['ans_set_valid']) > 1,  # если больше 1 значения
    final['ans_set_valid'].astype(str).apply(lambda x: x.split(';')[0]),  # вытащим первый номер
    final['ans_set_valid']  # оставим также
)

In [23]:
final

,text_id,markuper,markup_dt,subject1_mark,subject2_mark,subject3_mark,subject4_mark,ans_set_mark,subject1_valid,subject2_valid,subject3_valid,subject4_valid,ans_set_valid
0,text_id1,markuper1,2021-06-01 12:00:01,mq1,mq2,mq1,mq3,1,mq3,mq2,mq2,mq3,nan
1,text_id1,markuper3,2021-06-01 12:00:12,mq2,mq2,mq3,mq1,2,mq3,mq2,mq2,mq3,nan
2,text_id1,markuper2,2021-06-01 12:00:33,mq2,mq2,mq2,mq3,3,mq3,mq2,mq2,mq3,nan
3,text_id2,markuper3,2021-06-01 12:00:04,mq3,mq3,mq1,mq1,1,NaN,NaN,NaN,NaN,2
4,text_id2,markuper2,2021-06-01 12:00:05,mq2,mq3,mq1,mq3,2,NaN,NaN,NaN,NaN,2
5,text_id2,markuper1,2021-06-01 12:00:06,mq1,mq2,mq1,mq1,3,NaN,NaN,NaN,NaN,2
6,text_id3,markuper2,2021-06-01 12:00:07,mq2,mq3,mq1,mq3,1,mq2,mq3,mq1,mq1,nan
7,text_id3,markuper1,2021-06-01 12:00:08,mq2,mq2,mq1,mq1,2,mq2,mq3,mq1,mq1,nan
8,text_id3,markuper3,2021-06-01 12:00:09,mq3,mq3,mq1,mq1,3,mq2,mq3,mq1,mq1,nan
9,text_id4,markuper2,2021-06-01 12:00:10,mq2,mq3,mq1,mq3,1,NaN,NaN,NaN,NaN,1


In [24]:
final['ans_set_mark'] = final['ans_set_mark'].astype(float)
final['ans_set_valid'] = final['ans_set_valid'].astype(float)

In [25]:
# получаем таким образом итоговые значение по каждому тексту, некоторые будут с nan но их удалим позже
final['subject1_final'] = np.where(
    final['ans_set_valid'] == final['ans_set_mark'], 
    final['subject1_mark'],
    final['subject1_valid']
)
final['subject2_final'] = np.where(
    final['ans_set_valid'] == final['ans_set_mark'], 
    final['subject2_mark'],
    final['subject2_valid']
)
final['subject3_final'] = np.where(
    final['ans_set_valid'] == final['ans_set_mark'], 
    final['subject3_mark'],
    final['subject3_valid']
)
final['subject4_final'] = np.where(
    final['ans_set_valid'] == final['ans_set_mark'], 
    final['subject4_mark'],
    final['subject4_valid']
)

In [26]:
# вот тут очищаем данные
result_answers = final.dropna(
    subset=['text_id', 'subject1_final', 'subject2_final', 'subject3_final', 'subject4_final']
    ).drop_duplicates(subset=['text_id']
                      )[['text_id', 'subject1_final', 'subject2_final', 'subject3_final', 'subject4_final']]

## Финальная витрина правильных ответов

In [27]:
result_answers

,text_id,subject1_final,subject2_final,subject3_final,subject4_final
0,text_id1,mq3,mq2,mq2,mq3
4,text_id2,mq2,mq3,mq1,mq3
6,text_id3,mq2,mq3,mq1,mq1
9,text_id4,mq2,mq3,mq1,mq3


# Решаем задачу №2 - считаем среднюю точность разметчиков

In [28]:
quality = markup.merge(result_answers, how='inner', on='text_id') # соединиям данные и маркап, чтобы потом посчитать качество разметчиков

In [29]:
# ставим один если совпал ответ маркапера и итогового значения после валидации и 0 если не совпали
quality['subject1_right'] = np.where(quality['subject1'] == quality['subject1_final'], 1, 0)
quality['subject2_right'] = np.where(quality['subject2'] == quality['subject2_final'], 1, 0)
quality['subject3_right'] = np.where(quality['subject3'] == quality['subject3_final'], 1, 0)
quality['subject4_right'] = np.where(quality['subject4'] == quality['subject4_final'], 1, 0)

In [30]:
# делим на 4 чтобы получить процент правильных ответов
quality['score'] = (
    (quality['subject1_right'] + quality['subject2_right'] + quality['subject3_right'] + quality['subject4_right']) / 4
)

In [31]:
result = quality.merge(valid[['text_id', 'ans_set']], 
                       how='left', 
                       on='text_id', 
                       suffixes=['_mark', '_valid'])

In [32]:
result['ans_set_mark'] = result['ans_set_mark'].astype(str)

In [33]:
result['ans_set_valid'].fillna('0', inplace=True)

In [34]:
# проверяем выбирал ли валидатор значения валидатор как правильные у более чем 1 разметчика, если выбрал, то ставим 1 в графу эффективность
result['score'] = np.where(
    result.apply(lambda x: x.ans_set_mark in x.ans_set_valid, axis=1),  # если ответ разметчика валидатор посчитал верным, то ставим 1
    1, result['score']
)

In [35]:
markupers_df = result.groupby('markuper')['score'].mean().reset_index() # итоговые значения эффективности разметк для markuper

## Финальный ответ по задаче №2

In [36]:
markupers_df

,markuper,score
0,markuper1,0.625
1,markuper2,0.875
2,markuper3,0.500


In [37]:
markupers_df.to_csv('markupers_quality.csv', index=False)